In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.utils import SAVED_PICS_PATH, keras_dataset_image_preprocessing, DENSE_LAYER_NAME, CONV_LAYER_NAME, ALPHAS
from src.utils import SAVED_CLASSIFIER_PATH_MNIST, SAVED_CLADEC_PATH_MNIST, SAVED_REFAE_PATH_MNIST
from src.cladec import ClaDec, RefAE

_, (test_images, test_labels_c) = keras_dataset_image_preprocessing(keras.datasets.mnist)
SAVED_EVAL_PICS_PATH = SAVED_PICS_PATH.joinpath('fashion_mnist')

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_MNIST)

2022-07-09 14:22:49.426358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:22:49.462031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:22:49.462383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-09 14:22:49.462901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
decoder_dense = ClaDec.create_128_dense_decoder()
decoder_conv = ClaDec.create_64_conv_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')

    cladec_dense = ClaDec.create_from_weights(cladec_path.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                              classifier, DENSE_LAYER_NAME, alpha / 100, decoder_dense)
    print("Evaluating ", cladec_path.joinpath(DENSE_LAYER_NAME))
    cladec_dense.evaluate(test_images, test_labels_c)
    cladec_conv = ClaDec.create_from_weights(cladec_path.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                             classifier, CONV_LAYER_NAME, alpha / 100, decoder_conv)
    print("Evaluating ", cladec_path.joinpath(CONV_LAYER_NAME))
    cladec_conv.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_dense


2022-07-09 14:22:51.960751: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


313/313 [==============================] - 3s 6ms/step - loss: 0.0217 - reconstruction_loss: 0.0214 - classification_loss: 0.2416
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0018 - reconstruction_loss: 0.0018 - classification_loss: 0.0481
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_dense
313/313 [==============================] - 2s 5ms/step - loss: 0.0223 - reconstruction_loss: 0.0217 - classification_loss: 0.0491
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 0.0023 - reconstruction_loss: 0.0019 - classification_loss: 0.0441
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/50/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 0.0613 - reconstruction_loss: 0.0789 - classification_loss: 0.0455
Evaluating  /home/tom/Uni_De

In [5]:
# load RefAE model
refae_dense = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                        cladec_dense)
refae_conv = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                       cladec_conv)
# get reference reconstructions
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME))
refae_dense.evaluate(test_images)
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME))
refae_conv.evaluate(test_images)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_dense
313/313 [==============================] - 1s 4ms/step - reconstruction_loss: 0.0032
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_conv
313/313 [==============================] - 2s 4ms/step - reconstruction_loss: 5.4720e-04


0.0005471963668242097